Time series forecast of only one page with Facebook Prophet library. I borrow codes from PROPHET web site Quick Start page, of which address is https://facebookincubator.github.io/prophet/docs/quick_start.html#python-api
and Bojan Tunguz's Kernel Forecast example w/ Prophet - median, of which address is https://www.kaggle.com/tunguz/forecast-example-w-prophet-median

Prophet follows the sklearn model API. Facebook team creates an instance of the **Prophet** class and then call its** fit** and **predict** methods.
The input to Prophet is always a **dataframe with two columns**: **ds and y**. The ds (datestamp) column must contain a date or datetime (either is fine). The y column must be numeric, and represents the measurement we wish to forecast.


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from fbprophet import Prophet

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv("../input/DEXKOUS.csv")
df = df.rename(columns={'DATE': 'ds', 'DEXKOUS': 'y'}) # DEXKOUS is value of Korean won regarding to 1 US dollor on each date
df.head(100)

In [ ]:
# In row 14 and 84, the value of y is '.'. So I delete these row, by using codes
df=df[df['y'] != '.']
df.head(100)

Fitting the model by instantiating a new Prophet object. Any settings to the forecasting procedure are passed into the constructor. Then you call its **fit** method and pass in the historical dataframe. Fitting should take 1-5 seconds.

In [ ]:
# Python
m = Prophet()
m.fit(df);


Predictions are then made on a dataframe with a column ds containing the dates for which a prediction is to be made. You can get a suitable dataframe that extends into the future a specified number of days using the helper method *Prophet.make_future_dataframe*. By default it will also include the dates from the history, so we will see the model fit as well.

In [ ]:
future = m.make_future_dataframe(periods=60)
future.tail()

The predict method will assign each row in **future**** a predicted value which it names **yhat****. If you pass in historical dates, it will provide an in-sample fit. The **forecast**** object here is a new dataframe that includes a column yhat with the forecast, as well as columns for components and uncertainty intervals.

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)


You can plot the forecast by calling the **Prophet.plot**** method and passing in your forecast dataframe.

In [ ]:
m.plot(forecast);


If you want to see the forecast components, you can use the** Prophet.plot_components**** method. By default you’ll see the trend, yearly seasonality, and weekly seasonality of the time series. If you include **holidays**, you’ll see those here, too.

In [ ]:
m.plot_components(forecast);


**Outliers****

There are two main ways that outliers can affect Prophet forecasts. Here we make a forecast on the logged Wikipedia visits to the R page from before, but with a block of bad data:

In [ ]:
future = m.make_future_dataframe(periods=1260)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)
m.plot(forecast);

The trend forecast seems reasonable, but the uncertainty intervals seem way too wide. Prophet is able to handle the outliers in the history, but only by fitting them with trend changes. The uncertainty model then expects future trend changes of similar magnitude.
The best way to handle outliers is to remove them - Prophet has no problem with missing data. If you set their values to NA in the history but leave the dates in future, then Prophet will give you a prediction for their values.

In [ ]:
df.loc[(df['ds'] > '2008-01-01') & (df['ds'] < '2009-09-01'), 'y'] = None
model = Prophet().fit(df)
model.plot(model.predict(future));